# Geospatial Python
## Accessing satellite imagery using Python
Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Instruction: https://carpentries-incubator.github.io/geospatial-python/05-access-data.html

Objectives:
* Search public [SpatioTemporal Asset Catalog (STAC)](https://github.com/radiantearth/stac-api-spec/tree/release/v1.0.0) repositories of satellite imagery using Python.
* Inspect search result’s metadata.
* Download (a subset of) the assets available for a satellite scene.
* Open satellite imagery as raster data and save it to disk.

Before executing the code cells, be sure to **fill in the blanks** by replacing the "_____" as appropriate

### SpatioTemporal Asset Catalog (STAC) specification 

- Terabytes of data products are added daily to the satellite collections 
- Downloading these data to your local hard drive is not practical 
- Graphical User Interfaces (GUIs) are often available allowing online exploration  
  - E.g https://dataspace.copernicus.eu/browser/ 
- Drawbacks to manually downloading: 
  - Easy to miss items	 
  - Not easily reproducible 



- Benefits of retrieving data programmatically:
  * More reliable
  * Scalable 
  * Reproducible	

In [ ]:
# First import necessary libraries
import rioxarray # to open and download remote raster data
from pystac_client import Client# to query STAC API endpoint

from shapely.geometry import Point # to create a point 
import geojson # to parse spatial data
import folium # to create an interactive map
from folium.plugins import Draw # to allow drawing

# Create a variable to determine if the notebook is being run locally
local_run=False
if local_run:
    # this package is problematic on remote computers
    from localtileserver import TileClient, get_folium_tile_layer # to visualize the geotiff

In [ ]:
# Create an interactive map for use in creating a point of interest (POI)

# Start by defining a point to center the map on
center_coord = [40.60104027382292, -105.09137099497742] # fort collins

# Create the map
m  = folium.Map(center_coord, zoom_start=5)

# Add drawing controls
draw = Draw(export=True)
draw.add_to(m)

# Show the map
"_____"

# Use the drawing tools to create a POI
# Click the POI to get the coordinates (e.g numbers between '[' and ']'.

In [ ]:
# Go to https://radiantearth.github.io/stac-browser/
# Search for the "earth search"
# Click the link

# Then from the Earth Search page, click the "Source" button, and copy the URL from the text field

# Paste the Earth Search STAC catalog API URL below
api_url = "_____"

# Open the API
client = Client.open("_____")

In [ ]:
# Perform a metadata search 
# limited to 10 results from Sentinel-2, Level 2A, to retrieve Cloud Optimized GeoTiffs (COGs)

# Store a variable pointing to the collection of interest
# Note: Collection ID is taken from Sentinel-2 Level 2A - https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-c1-l2a
collection = "_____" 
'''
This collection includes Sentinel-2 data products 
pre-processed at level 2A (bottom-of-atmosphere reflectance) 
and saved in Cloud Optimized GeoTIFF (COG) format
'''

# Create a point to search from
# Note: values are in format x (long), y (lat) https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html
point = Point("_____","_____",)  # From interactive map above 
# Alternatively use https://www.google.com/maps > search for place > right click to access lat and lng
# lat (y) usually displayed before lng x, lng goes from -180 (west of Greenwich) - 0 - 180

# Perform the search
search = client.search(
    collections=[collection],
    intersects="_____",
    max_items=10,
)
# show the number of scenes (i.e. the portion of the footage recorded by the satellite)
print(search.matched())

In [ ]:
# Store the metadata of the search results
items = search.item_collection()

In [ ]:
# Get the length of items
print(len("_____"))

In [ ]:
# Loop over the items to see there ids
for item in items:
    print("_____")

In [ ]:
# Inspect the metadata associated with the first item of the search results
item = items["_____"]
print(item.datetime)
print(item.geometry)
print(item.properties)

In [ ]:
'''
EXERCISE: Search the sentinel-2-l2a collection for all the available scenes that satisfy the following criteria: 
- intersect a provided bounding box (use ±0.01 deg in lat/lon from the previously defined point); 
- have been recorded between 20 March 2020 and 30 March 2020; 
- have a cloud coverage less than 15. Note: the eo extension (https://github.com/stac-extensions/eo) is implemented in some collections allowing it to be queried against

* Get the count
* Save the results to json
'''
bbox = point.buffer("_____").bounds

search = client.search(
    collections=[collection],
    bbox=bbox,
    datetime="2023-03-20/2024-03-30",
    query=["eo:cloud_cover<15"]
)
print(search.matched())
items = search.item_collection()
items.save_object("search.json") # json file saved alongside notebook

## Access the assets


In [ ]:
# Get the first item's assets
assets = items[0].assets  
# print each attribute name using keys()
print(assets."_____"())

In [ ]:
# Print a minimal description of the available assets
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

In [ ]:
# Show one metadata value
print(assets["thumbnail"])

# Show the 'href' attribute value
print(assets["thumbnail"]."_____")

In [ ]:
# Open 'nir' with the rioxarray library, use an overview level of 3 to reduce the file size
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio("_____", overview_level="_____")
print(nir)

In [ ]:
# save whole image to disk - this may take awhile
# Actually, let's not, as this is a large file
# nir.rio.to_raster("nir.tif")

In [ ]:
# Let's show the boundary box of the whole tiff on an interactive map
# Interactive maps generally expect data that's in CRS 4326
# We'll need to convert our boundary box to this CRS

# Some extra modules and a library are required for this
from shapely.geometry import box # To create a box
from shapely.ops import transform # The shapely transform module  
import pyproj # A reprojection library

# Create the transformer
project = pyproj.Transformer.from_crs(nir.rio.crs.to_epsg(), "_____", always_xy=True).transform

# Create our boundary box
"_____" = box(*nir.rio.bounds())

# Apply the transformation
bbox_transformed = transform(project, "_____")

bbox_transformed


In [ ]:
# Let's use an interctive map to create an area of interest (AOI) for use in subsetting our data. 

# Create and center our map to the our POI
m = folium.Map(location=[point.y,point.x], zoom_start=12)

# Create tiles client
if local_run:
    # Geotiff files will not show in our web browser, but we can convert them on-the-fly
    # We'll use a library that goes one step further, and generates map tiles.
    # Map tiles allow us to only load in the part of the image we are intersted in when moving around the map
    nir_tiles = TileClient(nir_href) 
    nir_layer = get_folium_tile_layer(nir_tiles, name='nir') # create elevation tile layer
    nir_layer.add_to(m)

# show the boundary box of the whole tiff
folium.GeoJson(bbox_transformed,
    style_function=lambda feature: {
        "color": "red",
    }).add_to(m)


draw = Draw(export=True)
draw.add_to(m)

# add a layer control to toggle layers
folium.LayerControl().add_to(m)
m

# Use the draw rectangle tool to create a shape that overlaps a portion of the boundary

In [ ]:
# Save a portion of the raster to disk

# Copy the geojson from the drawn polygon (click the shape, and copy the text starting from '{"type":"Polygon"', up until the last '}').
geom='''"_____"'''
geojson.loads(geom)
cropping_geometries = [geojson.loads(geom)] # converts to list

# Use .rio.clip to subset the raster data
# Doc: https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray-rio-accessors
nir_sub = nir.rio.clip(geometries=cropping_geometries, crs=4326)
nir_sub.rio.to_raster("_____")

In [ ]:
# We'll also show the boundary box of the subset tiff on an interactive map
# This too will require reprojecting our boundary box
bbox_clip = box(*rioxarray.open_rasterio("_____").rio.bounds())
bbox_clip_transformed = transform(project, bbox_clip)

bbox_clip_transformed

In [ ]:
# Create an interctive map showing our AOI and subsetted raster

m = folium.Map(location=[point.y,point.x], zoom_start=12)

if local_run:
    nir_tiles = TileClient("_____") # create tiles client
    nir_layer = get_folium_tile_layer(nir_tiles, name='nir')
    nir_layer.add_to(m)

# show the boundary box of the whole tiff
folium.GeoJson(bbox_transformed,
    style_function=lambda feature: {
        "color": "red",
    }).add_to(m)

# show the boundary box of the clipped tiff
folium.GeoJson(bbox_clip_transformed,
    style_function=lambda feature: {
        "color": "green",
    }).add_to(m)

# show our drawn polygon
folium.GeoJson(geom).add_to(m)



folium.LayerControl().add_to(m)

m